## Raw data import from GWOSC

In [1]:
import pandas as pd
import wget
import os
from pycbc.frame import read_frame
import scipy as sp

In [144]:
ifo = ['H', 'L']
no_segments = 1450
t_start = 1164643217 # Start of O2 run is at GPS time 1164556817
t_stop = t_start + no_segments*4096 # End of 02 run is at GPS time 1187733618, or 5658 4096 s segments.

In [145]:
df = {}

for i in ifo:

    url = 'https://www.gw-openscience.org/archive/links/O2_4KHZ_R1/%s1/%s/%s/html/' %(i, t_start, t_stop)
    print(url)

    df_tmp = pd.read_html(url, header=1)[0]

    # Ensuring data quality and no injections
    DATA = df_tmp['DATA'] >= 100
    CAT1 = df_tmp['CBC_CAT1'] == 100
    CAT2 = df_tmp['CBC_CAT2'] == 100
    CAT3 = df_tmp['CBC_CAT3'] == 100
    NO_CBC = df_tmp['NO_CBC_HW_INJ'] == 100
    NO_BURST = df_tmp['NO_BURST_HW_INJ'] == 100
    NO_DET = df_tmp['NO_DETCHAR_HW_INJ'] == 100
    index = DATA & CAT1 & CAT2 & CAT3 & NO_CBC & NO_BURST & NO_DET

    df[i] = df_tmp[index]
    df[i] = df[i].reset_index(drop=True)

# Find data segments from both detectors that uphold quality requirements
df_concat = pd.concat([df['H'], df['L']])
df_filtered = df_concat[df_concat.duplicated(subset='Unnamed: 0')]
df_filtered = df_filtered.reset_index(drop=True)

https://www.gw-openscience.org/archive/links/O2_4KHZ_R1/H1/1164643217/1170582417/html/
https://www.gw-openscience.org/archive/links/O2_4KHZ_R1/L1/1164643217/1170582417/html/


In [148]:
directory = '/media/jacob/GRAS2BackupV2/Waveform_test'

prfx_strt = 1163919360
prfx_diff = 256*4096
prfx_no = 23
prefixes = np.arange(prfx_strt, prfx_strt+prfx_no*prfx_diff, prfx_diff)

j = 1

for i in df_filtered['Unnamed: 0']:
    
    print(j)
    j += 1
    
    prfx_sub = prefixes-i
    prefix = prefixes[np.where(prfx_sub <= 0)[0][-1]]
    
    urlH = ('https://www.gw-openscience.org/archive/data/O2_4KHZ_R1/%s/H-H1_' %(prefix) +
           'GWOSC_O2_4KHZ_R1-%s-4096.gwf' %(i))
    urlL = ('https://www.gw-openscience.org/archive/data/O2_4KHZ_R1/%s/L-L1_' %(prefix) +
           'GWOSC_O2_4KHZ_R1-%s-4096.gwf' %(i))
    
    wget.download(urlH, directory+'/Raw_data')
    wget.download(urlL, directory+'/Raw_data')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [149]:
j = 1

for filename in os.listdir(directory+'/Raw_data'):
    
    print(j)
    j += 1
    
    if 'H-H1' in filename:
        channel = 'H1:GWOSC-4KHZ_R1_STRAIN'
    else:
        channel = 'L1:GWOSC-4KHZ_R1_STRAIN'
    
    fname = directory+'/Raw_data/'+filename
    print(fname)
    print(channel)
    data_tmp = read_frame(fname, channel)
    times = data_tmp.sample_times
    
    for i in range(128):
        
        filename_split = filename.split('-')
        data_slice = data_tmp.time_slice(times[0]+i*32., times[0]+(1.+i)*32.)
        data_slice.save(directory+'/Sliced_data/%s_%s_%s.npy' %(filename_split[0], filename_split[2], i+1))

1
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/L-L1_GWOSC_O2_4KHZ_R1-1165144064-4096.gwf
L1:GWOSC-4KHZ_R1_STRAIN
2
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/H-H1_GWOSC_O2_4KHZ_R1-1164689408-4096.gwf
H1:GWOSC-4KHZ_R1_STRAIN
3
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/H-H1_GWOSC_O2_4KHZ_R1-1164693504-4096.gwf
H1:GWOSC-4KHZ_R1_STRAIN
4
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/H-H1_GWOSC_O2_4KHZ_R1-1164701696-4096.gwf
H1:GWOSC-4KHZ_R1_STRAIN
5
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/H-H1_GWOSC_O2_4KHZ_R1-1164705792-4096.gwf
H1:GWOSC-4KHZ_R1_STRAIN
6
/media/jacob/GRAS2BackupV2/Waveform_test/Raw_data/H-H1_GWOSC_O2_4KHZ_R1-1164713984-4096.gwf
H1:GWOSC-4KHZ_R1_STRAIN


KeyboardInterrupt: 